<a href="https://colab.research.google.com/github/gopinathak-geek/novozymes-enzyme-stability-prediction/blob/main/NovoModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd

In [3]:
train_csv = "https://raw.githubusercontent.com/gopinathak-geek/novozymes-enzyme-stability-prediction/main/data/train.csv"
train_updates_csv = "https://raw.githubusercontent.com/gopinathak-geek/novozymes-enzyme-stability-prediction/main/data/train_updates_20220929.csv"
test_csv = "https://raw.githubusercontent.com/gopinathak-geek/novozymes-enzyme-stability-prediction/main/data/test.csv"

train_df = pd.read_csv(train_csv)
train_updates_df = pd.read_csv(train_updates_csv)
test_df = pd.read_csv(test_csv)

In [4]:
seq_ids_to_delete = train_updates_df[train_updates_df["pH"].isnull()]["seq_id"].values
seq_ids_to_replace = train_updates_df[train_updates_df["pH"].notnull()]["seq_id"].values

In [5]:
train_df.drop(train_df[train_df.seq_id.isin(seq_ids_to_delete)].index, inplace=True)

In [6]:
train_df.loc[train_df.seq_id.isin(seq_ids_to_replace), ["pH", "tm"]] = train_updates_df[train_updates_df.seq_id.isin(seq_ids_to_replace)][["pH", "tm"]].values

In [7]:
null_seq_ids = train_df[train_df["pH"].isnull()]["seq_id"].values

In [8]:
train_df.drop(train_df[train_df.seq_id.isin(null_seq_ids)].index, inplace=True)

In [9]:
train_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28695 entries, 0 to 31389
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   seq_id            28695 non-null  int64  
 1   protein_sequence  28695 non-null  object 
 2   pH                28695 non-null  float64
 3   data_source       27727 non-null  object 
 4   tm                28695 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.3+ MB


In [10]:
train_df.describe().transpose()


,count,mean,std,min,25%,50%,75%,max
seq_id,28695.0,15631.432166,9235.099078,0.00,7447.5,15443.0,23758.5,31389.0
pH,28695.0,6.872467,0.793184,1.99,7.0,7.0,7.0,11.0
tm,28695.0,51.385604,12.076609,25.10,43.7,48.8,54.6,130.0


In [11]:
import sys
print (sys.version)


3.7.15 (default, Oct 12 2022, 19:14:55) 
[GCC 7.5.0]


In [12]:
import torch
import random
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from scipy import stats
import torchvision.transforms as T


In [13]:
def getMolecularWeight(aminoacid):
  if aminoacid == "A": return 89
  elif aminoacid == "R": return 174
  elif aminoacid == "N": return 132
  elif aminoacid == "D": return 133
  elif aminoacid == "B": return 133
  elif aminoacid == "C": return 121
  elif aminoacid == "Q": return 146
  elif aminoacid == "E": return 147
  elif aminoacid == "Z": return 147
  elif aminoacid == "G": return 75
  elif aminoacid == "H": return 155
  elif aminoacid == "I": return 131
  elif aminoacid == "L": return 131
  elif aminoacid == "K": return 146
  elif aminoacid == "M": return 149
  elif aminoacid == "F": return 165
  elif aminoacid == "P": return 115
  elif aminoacid == "S": return 105
  elif aminoacid == "T": return 119
  elif aminoacid == "W": return 204
  elif aminoacid == "Y": return 181
  elif aminoacid == "V": return 117
  else: return 0

In [14]:
def proteinSequenceToAmioAcidMolecularWeightWithPh(data):
  amionAcidSequence = []
  img = []
  label = []
  for index, row in data.iterrows():
    ph = row['pH']
    tm = row['tm']
    molecularWeightOfAminoAcidInSequence = []
    pHofProteinSequence = []
    for aminoacid in (row['protein_sequence']):
      molecularWeight = getMolecularWeight(aminoacid)
      molecularWeightOfAminoAcidInSequence.append(molecularWeight/110)
      pHofProteinSequence.append(ph)
    #a = torch.FloatTensor(np.array([molecularWeightOfAminoAcidInSequence,pHofProteinSequence]))
    #a = a[None, :]
    #a = preprocess(np.array([molecularWeightOfAminoAcidInSequence,pHofProteinSequence]))
    a = np.array([molecularWeightOfAminoAcidInSequence,pHofProteinSequence])
    a = a[None, :]
    img.append(a)
    label.append(tm)
    #amionAcidSequence.append((a, tm))
    #break
  return [img, label]

In [15]:
img, label = proteinSequenceToAmioAcidMolecularWeightWithPh(train_df)

In [16]:
img[0]

array([[[0.80909091, 0.80909091, 0.80909091, 0.80909091, 1.32727273,
         0.80909091, 0.80909091, 0.80909091, 1.19090909, 0.80909091,
         1.19090909, 1.19090909, 0.68181818, 1.33636364, 0.80909091,
         1.04545455, 1.33636364, 1.06363636, 1.06363636, 1.20909091,
         1.19090909, 1.85454545, 1.19090909, 1.04545455, 0.80909091,
         0.68181818, 1.85454545, 1.58181818, 1.32727273, 1.04545455,
         1.5       , 1.58181818, 1.06363636, 1.5       , 1.58181818,
         1.19090909, 1.33636364, 1.58181818, 1.32727273, 0.68181818,
         1.20909091, 0.68181818, 1.06363636, 1.19090909, 1.06363636,
         0.68181818, 1.35454545, 1.19090909, 1.32727273, 1.20909091,
         0.80909091, 0.68181818, 1.20909091, 1.20909091, 1.04545455,
         1.20909091, 1.06363636, 1.08181818, 1.40909091, 0.68181818,
         0.80909091, 1.33636364, 1.19090909, 1.32727273, 0.80909091,
         1.5       , 1.06363636, 1.58181818, 1.5       , 0.80909091,
         0.95454545, 1.33636364, 1

In [17]:
import torchvision.transforms as T
from PIL import Image

preprocess = T.Compose([
   T.Resize((2,10)),
])

def change_shape(data):
    torch_img = []
    for im in (data):
        im = torch.FloatTensor(im)
        #transform = T.ToPILImage()
        #img = transform(im)
        x = preprocess(im)
        x = x.cpu().detach().numpy()
        torch_img.append(x)
    return torch_img

In [18]:
img = change_shape(img)

In [19]:
class TrainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [20]:
train_data = TrainData(torch.FloatTensor(img), torch.FloatTensor(label))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  """Entry point for launching an IPython kernel.


In [21]:
image, label = train_data[2]
print(image.shape, label)

torch.Size([1, 2, 10]) tensor(40.5000)


In [22]:
train_size = int(0.8 * len(train_data))
val_size = len(train_data) - train_size
training_data, validation_data = torch.utils.data.random_split(train_data, [train_size, val_size])

In [23]:
BATCH_SIZE = 128
training_loader = DataLoader(dataset=training_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
validation_loader = DataLoader(dataset=validation_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

In [24]:
class NovoNet(nn.Module):
    def __init__(self, name=None):
        super(NovoNet, self).__init__()
        if name:
            self.name = name
        self.conv1 = nn.Conv2d(1, 10, 2) 
        #self.pool = nn.MaxPool2d((3, 2), stride=(2, 2))
        self.fc1 = nn.Linear(10 * 9 * 1, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)
        
        # compute the total number of parameters
        total_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        print(self.name + ': total params:', total_params)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = x.view(-1, 9 * 1 * 10)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [25]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [26]:
net = NovoNet(name='Novonet')
net.to(device)

Novonet: total params: 21219


NovoNet(
  (conv1): Conv2d(1, 10, kernel_size=(2, 2), stride=(1, 1))
  (fc1): Linear(in_features=90, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=1, bias=True)
)

In [27]:
LEARNING_RATE = 0.001
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=LEARNING_RATE)

In [28]:
EPOCHS = 500
for epoch in range(EPOCHS):
    training_score = []
    for img, label in training_loader:
      net.train()
      img, label = img.to(device), label.to(device)
      label = label.unsqueeze(1)
    
      #==========Forward pass===============
      preds = net(img)
      loss = criterion(preds, label)
      #==========backward pass==============

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()


      train_result = stats.spearmanr(preds.detach().cpu().numpy(), label.cpu().numpy())
      training_score.append(train_result.correlation)
        
    validation_score = []
    for img, label in validation_loader:
      net.eval()
      img, label = img.to(device), label.to(device)
      label = label.unsqueeze(1)
      val_preds = net(img)
      val_result = stats.spearmanr(val_preds.detach().cpu().numpy(), label.cpu().numpy())
      validation_score.append(val_result.correlation)

    print(f'{epoch+1:03} EPOCH scores -  Training score : {np.mean(training_score):.5f} | Validation score : {np.mean(validation_score):.5f}')

001 EPOCH scores -  Training score : 0.02487 | Validation score : -0.02379
002 EPOCH scores -  Training score : -0.00311 | Validation score : -0.00557
003 EPOCH scores -  Training score : 0.01849 | Validation score : 0.01691
004 EPOCH scores -  Training score : 0.03546 | Validation score : 0.00255
005 EPOCH scores -  Training score : 0.03308 | Validation score : 0.02032
006 EPOCH scores -  Training score : 0.03709 | Validation score : 0.00720
007 EPOCH scores -  Training score : 0.03746 | Validation score : 0.01057
008 EPOCH scores -  Training score : 0.03923 | Validation score : 0.01775
009 EPOCH scores -  Training score : 0.04489 | Validation score : 0.01535
010 EPOCH scores -  Training score : 0.04773 | Validation score : 0.01181
011 EPOCH scores -  Training score : 0.05035 | Validation score : 0.03201
012 EPOCH scores -  Training score : 0.05905 | Validation score : 0.03905
013 EPOCH scores -  Training score : 0.06000 | Validation score : 0.04758
014 EPOCH scores -  Training score 

In [29]:
test_df.head()

,seq_id,protein_sequence,pH,data_source
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes


In [30]:
test_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2413 entries, 0 to 2412
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   seq_id            2413 non-null   int64 
 1   protein_sequence  2413 non-null   object
 2   pH                2413 non-null   int64 
 3   data_source       2413 non-null   object
dtypes: int64(2), object(2)
memory usage: 75.5+ KB


In [31]:
test_df.describe().transpose()


,count,mean,std,min,25%,50%,75%,max
seq_id,2413.0,32596.0,696.717422,31390.0,31993.0,32596.0,33199.0,33802.0
pH,2413.0,8.0,0.000000,8.0,8.0,8.0,8.0,8.0


In [32]:
seq_id = test_df['seq_id']
seq_id

0       31390
1       31391
2       31392
3       31393
4       31394
        ...  
2408    33798
2409    33799
2410    33800
2411    33801
2412    33802
Name: seq_id, Length: 2413, dtype: int64

In [33]:
def testProteinSequenceToAmioAcidMolecularWeightWithPh(data):
  amionAcidSequence = []
  img = []
  for index, row in data.iterrows():
    ph = row['pH']
    molecularWeightOfAminoAcidInSequence = []
    pHofProteinSequence = []
    for aminoacid in (row['protein_sequence']):
      molecularWeight = getMolecularWeight(aminoacid)
      molecularWeightOfAminoAcidInSequence.append(molecularWeight/110)
      pHofProteinSequence.append(ph)
    #a = torch.FloatTensor(np.array([molecularWeightOfAminoAcidInSequence,pHofProteinSequence]))
    #a = a[None, :]
    #a = preprocess(np.array([molecularWeightOfAminoAcidInSequence,pHofProteinSequence]))
    a = np.array([molecularWeightOfAminoAcidInSequence,pHofProteinSequence])
    a = a[None, :]
    img.append(a)
    #amionAcidSequence.append((a, tm))
    #break
  return img

In [34]:
test_img = testProteinSequenceToAmioAcidMolecularWeightWithPh(test_df)

In [35]:
test_img[0]

array([[[1.06363636, 1.04545455, 1.06363636, 1.2       , 1.04545455,
         1.33636364, 1.04545455, 1.20909091, 0.80909091, 1.08181818,
         0.95454545, 1.06363636, 1.33636364, 1.2       , 1.06363636,
         0.80909091, 1.33636364, 1.32727273, 1.08181818, 0.68181818,
         0.95454545, 0.68181818, 1.20909091, 0.95454545, 1.32727273,
         0.95454545, 1.20909091, 1.04545455, 1.19090909, 1.32727273,
         0.80909091, 1.20909091, 1.19090909, 1.33636364, 1.06363636,
         1.32727273, 0.68181818, 1.32727273, 0.95454545, 0.80909091,
         1.19090909, 1.04545455, 1.5       , 1.20909091, 1.06363636,
         1.20909091, 1.1       , 1.85454545, 0.80909091, 1.19090909,
         1.19090909, 1.1       , 1.32727273, 0.68181818, 0.80909091,
         1.04545455, 1.2       , 1.06363636, 1.19090909, 1.32727273,
         1.58181818, 1.06363636, 1.2       , 1.33636364, 1.32727273,
         1.08181818, 1.32727273, 1.2       , 0.95454545, 1.2       ,
         1.58181818, 1.20909091, 1

In [36]:
test_img = change_shape(test_img)

In [37]:
test_img

[array([[[1.0145454 , 1.2854549 , 0.9863636 , 1.3081816 , 0.99954134,
          0.828183  , 0.9327259 , 1.1636364 , 1.1381825 , 1.2663633 ],
         [8.        , 8.        , 8.        , 8.        , 8.        ,
          8.        , 8.        , 8.        , 8.        , 8.        ]]],
       dtype=float32),
 array([[[1.0145454 , 1.2854549 , 0.9863636 , 1.3081816 , 0.99954134,
          0.828183  , 0.9327259 , 1.1636364 , 1.1381825 , 1.2663633 ],
         [8.        , 8.        , 8.        , 8.        , 8.        ,
          8.        , 8.        , 8.        , 8.        , 8.        ]]],
       dtype=float32),
 array([[[1.0090909, 1.2      , 1.1227273, 1.0045455, 0.8818182,
          1.       , 0.8818182, 1.1363637, 1.1545455, 1.2727273],
         [8.       , 8.       , 8.       , 8.       , 8.       ,
          8.       , 8.       , 8.       , 8.       , 8.       ]]],
       dtype=float32),
 array([[[1.0145454 , 1.2854549 , 0.9863636 , 1.3081816 , 0.99954134,
          0.828183  , 0.93272

In [38]:
class TestData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [39]:
test_data = TestData(torch.FloatTensor(test_img))

In [40]:
test_loader = DataLoader(dataset=test_data, batch_size=1)

In [41]:
pred_list = []
net.eval()
with torch.no_grad():
    for img in test_loader:
        img = img.to(device)
        prediction = net(img)
        print(prediction)
        pred_list.append(prediction.cpu().numpy())
        #pred_list.append(prediction.cpu().numpy())
tm = [a.squeeze().tolist() for a in pred_list]

tensor([[59.7993]], device='cuda:0')
tensor([[59.7993]], device='cuda:0')
tensor([[61.6959]], device='cuda:0')
tensor([[59.7993]], device='cuda:0')
tensor([[59.7993]], device='cuda:0')
tensor([[59.7993]], device='cuda:0')
tensor([[59.7993]], device='cuda:0')
tensor([[59.7993]], device='cuda:0')
tensor([[59.7993]], device='cuda:0')
tensor([[59.7993]], device='cuda:0')
tensor([[59.7993]], device='cuda:0')
tensor([[59.7993]], device='cuda:0')
tensor([[59.7993]], device='cuda:0')
tensor([[61.6959]], device='cuda:0')
tensor([[59.7993]], device='cuda:0')
tensor([[59.7993]], device='cuda:0')
tensor([[59.7993]], device='cuda:0')
tensor([[59.7993]], device='cuda:0')
tensor([[59.7993]], device='cuda:0')
tensor([[59.7993]], device='cuda:0')
tensor([[59.7993]], device='cuda:0')
tensor([[59.7993]], device='cuda:0')
tensor([[59.7993]], device='cuda:0')
tensor([[59.7993]], device='cuda:0')
tensor([[59.7993]], device='cuda:0')
tensor([[59.7993]], device='cuda:0')
tensor([[59.7993]], device='cuda:0')
t

In [42]:
tm

[59.799346923828125,
 59.799346923828125,
 61.69590759277344,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 61.69590759277344,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.799346923828125,
 59.79934692382

In [43]:
df = pd.DataFrame({"seq_id":seq_id, "tm":tm })
df

,seq_id,tm
0,31390,59.799347
1,31391,59.799347
2,31392,61.695908
3,31393,59.799347
4,31394,59.799347
...,...,...
2408,33798,59.799347
2409,33799,59.799347
2410,33800,59.799347
2411,33801,59.799347


In [44]:
df.to_csv("submission.csv", index=False)